In [7]:
!pip install tf-agents[reverb]

In [11]:
from __future__ import absolute_import, division, print_function

import keras
import numpy as np
import tensorflow as tf
import reverb
import random

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers.py_driver import PyDriver
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.networks import sequential
from tf_agents.trajectories import time_step as ts
from tf_agents.policies.py_tf_eager_policy import PyTFEagerPolicy
from tf_agents.specs import tensor_spec
from tf_agents.specs.array_spec import BoundedArraySpec
from tf_agents.utils import common

In [12]:
fully_connected_layers: tuple[int, int] = (100, 50)
num_iterations: int = 20_000

initial_collect_steps: int = 100
collect_steps_per_iteration: int = 1

max_buffer_size: int = 10_000
batch_size: int = 64
learning_rate: float = 1e-3
log_interval: int = 200

num_eval_episodes: int = 10
eval_interval: int = 1_000

lane_length: int = 5
spawn_rate: float = 0.4
total_ticks: int = 50